In [1]:
__author__ = "Tim Cuddeback"

import yfpy, json, ast
from yfpy.query import YahooFantasySportsQuery
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Function that pulls data from Yahoo's Fantasy API and returns a sorted list of tuples with (team_id,current points)
league_id = '12514' # put your real league id here
game_id = "399" # put the game id here (game id's reflect the type of sport and the year)
game_code = "nfl" # put the game code here
season = "2020" # put the year of the current fantasy season
auth_dir = '' # put the location where you are storing the client_id/secret
working_dir = '' # put the location where you want excel file to be saved
excel_file_name = 'Points.xlsx' # Name your file after you set up the headers

# Query yffpy to get data
yahoo_query = YahooFantasySportsQuery(auth_dir, league_id, game_id=game_id, game_code=game_code, offline=False)

In [2]:
# Get standings data to get the points for the day

data1 = yahoo_query.get_league_matchups_by_week(1)
data2 = yahoo_query.get_league_matchups_by_week(2)
data3 = yahoo_query.get_league_matchups_by_week(3)
data4 = yahoo_query.get_league_matchups_by_week(4)
data5 = yahoo_query.get_league_matchups_by_week(5)
data6 = yahoo_query.get_league_matchups_by_week(6)
data7 = yahoo_query.get_league_matchups_by_week(7)
data8 = yahoo_query.get_league_matchups_by_week(8)
data9 = yahoo_query.get_league_matchups_by_week(9)
data10 = yahoo_query.get_league_matchups_by_week(10)
data11 = yahoo_query.get_league_matchups_by_week(11)
data12 = yahoo_query.get_league_matchups_by_week(12)
data13 = yahoo_query.get_league_matchups_by_week(13)
data14 = yahoo_query.get_league_matchups_by_week(14)
data15 = yahoo_query.get_league_matchups_by_week(15)
data16 = yahoo_query.get_league_matchups_by_week(16)
data17 = yahoo_query.get_league_matchups_by_week(17)

# .get_league_teams(self)
# .get_league_players(self)
# .get_league_draft_results(self)
# .get_league_matchups_by_week(self,chosen_week)
# .get_team_info(self,team_id)
# .get_team_standings(self,team_id) # has current "streak" within
# .get_player_stats_by_week(self, player_key, chosen_week="current")

[2021-04-12 14:56:11,889 DEBUG] [yfpy.query.query.query] Response (JSON): {'fantasy_content': {'xml:lang': 'en-US', 'yahoo:uri': '/fantasy/v2/game/399/metadata', 'game': [{'game_key': '399', 'game_id': '399', 'name': 'Football', 'code': 'nfl', 'type': 'full', 'url': 'https://football.fantasysports.yahoo.com/f1', 'season': '2020', 'is_registration_over': 1, 'is_game_over': 1, 'is_offseason': 1}], 'time': '17.993927001953ms', 'copyright': 'Data provided by Yahoo! and STATS, LLC', 'refresh_rate': '60'}}
[2021-04-12 14:56:11,892 DEBUG] [yfpy.query.query.query] Data fetched with query URL: https://fantasysports.yahooapis.com/fantasy/v2/game/399/metadata?format=json
[2021-04-12 14:56:11,893 DEBUG] [yfpy.query.query.query] Response (Yahoo fantasy data extracted from: "fantasy_content"): {'xml:lang': 'en-US', 'yahoo:uri': '/fantasy/v2/game/399/metadata', 'game': [{'game_key': '399', 'game_id': '399', 'name': 'Football', 'code': 'nfl', 'type': 'full', 'url': 'https://football.fantasysports.yaho

KeyError: 'scoreboard'

In [6]:
jsondata = []
for i in range(0,len(data2)):
    jsondata.append(json.loads(str(data2[i]['matchup'])))
jsondata
df = pd.DataFrame()
cleaned = []
for i in range(0,len(jsondata)):
    row = {"week" : jsondata[i]['week'],
           "team_id1" : jsondata[i]['teams'][0]['team']['managers']['manager']['manager_id'],
           "owner1" : jsondata[i]['teams'][0]['team']['managers']['manager']['nickname'],
           "team1" : jsondata[i]['teams'][0]['team']['name'],
           "points1" : jsondata[i]['teams'][0]['team']['team_points']['total'],
           "team_id2" : jsondata[i]['teams'][1]['team']['managers']['manager']['manager_id'],
           "owner2" : jsondata[i]['teams'][1]['team']['managers']['manager']['nickname'],
           "team2" : jsondata[i]['teams'][1]['team']['name'],
           "points2" : jsondata[i]['teams'][1]['team']['team_points']['total']}
    df1 = pd.DataFrame(row, index=[0])
    df = pd.concat([df1,df])

,week,team_id1,owner1,team1,points1,team_id2,owner2,team2,points2
0,2,6,Peter Billups,Poor Decisions 😢,197.44,7,Pat,Push Ups in Prison,146.18
0,2,5,Greg,Krispy Kareem 🍩,121.80,8,Shinn,Table scraps,119.76
0,2,4,Wesley,Beats by DeAndre,117.38,9,Chris,Jonesin' 4 A Win,109.70
0,2,2,ryan,FreshPrinceOfHelaire,112.32,10,Jeremy,Gronk's Gurley Chubb,113.22
0,2,1,Tim,Fantasy Sidelines,125.18,3,Kevin,Butker is my lube,105.26
